# Imports:

In [2]:
from CSIKit.reader import get_reader
from CSIKit.util import csitools
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

# All File names from directory : 

In [6]:
#internet dataset
#mypath = r"data/"
mypath = r"pcap_files/"
file_names = [f.split(".")[0] for f in listdir(mypath) if isfile(join(mypath, f))]

# Replace inf rows, inf numbers and convert pcap to csv amplitudes files :

In [7]:
for name in file_names:
    my_reader = get_reader(mypath + f"{name}.pcap")
    csi_data = my_reader.read_file(mypath + f"{name}.pcap", scaled=True)
    csi_matrix, no_frames, no_subcarriers = csitools.get_CSI(csi_data, metric="amplitude")
    arr = csi_matrix[:,:,0,0]
    
    indx = []
    for count,i in enumerate(arr):
        if len(set(i.flat)) == 1:
            indx.append(count)
        else:
            arr[count] = np.where(i == -np.inf, np.ma.masked_invalid(i).mean(),i)
            
    arr = np.delete(arr,indx,axis=0)
    df = pd.DataFrame(arr)
    df.to_csv(path_or_buf = fr"csv_files/csv_amplitude/{name}.csv" , index = False)